In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
import torchvision
from torchvision.transforms import v2
from tqdm.notebook import tqdm

import torchinfo
import optuna

import time
import gc

In [3]:
if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

print("Using", device)

Using cuda


# train_model and test_model functions

In [4]:
# ==============================================================================
# START OF test_model()
# ==============================================================================
def test_model(model,
               test_loader,
               device=torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'),
               name=''):
    model.eval()
    with torch.no_grad():
        test_count = 0
        test_correct_count = 0

        for _, (test_x, test_y) in enumerate(test_loader):
            test_x = test_x.float().to(device)
            test_y = test_y.long().to(device)

            logits = model(test_x).detach()
            y_hat = torch.argmax(logits, dim=-1)

            test_correct_count += torch.sum(y_hat == test_y, axis=-1)
            test_count += test_x.size(0)

        test_acc = test_correct_count / test_count
        print(name, 'Test acc: {:.3f}'.format(test_acc))
# ==============================================================================
# END OF test_model()
# ==============================================================================

# data_prep function

In [5]:
def data_prep(X,y,sub_sample,average,noise,channel_dropout,time_reverse):

    total_X = None
    total_y = None
    X = X[:,:,0:800]
    print('Shape of X after trimming:',X.shape)

    X_max, _ = torch.max(X.view(X.size(0), X.size(1), -1, sub_sample), axis=3)


    total_X = X_max
    total_y = y
    print('Shape of X after maxpooling:',total_X.shape)

    X_average = torch.mean(X.view(X.size(0), X.size(1), -1, average), axis=3)
    X_average = X_average + torch.normal(0.0, 0.5, X_average.shape)

    total_X = torch.cat((total_X, X_average), dim=0)
    total_y = torch.cat((total_y, y))
    print('Shape of X after averaging+noise and concatenating:',total_X.shape)

    for i in range(sub_sample):

        X_subsample = X[:, :, i::sub_sample] + \
                            (torch.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)

        total_X = torch.cat((total_X, X_subsample), dim=0)
        print(total_y.view(-1,1).shape)
        print(y.view(-1,1).shape)
        total_y = torch.cat((total_y, y))

    print('Shape of X after subsampling and concatenating:',total_X.shape)
    print('Shape of Y:',total_y.shape)

    if channel_dropout != 0:
        mask = (torch.rand(total_X.shape[0], total_X.shape[1]) >= channel_dropout).unsqueeze(2)
        X_dropout = mask * total_X
        total_X = torch.cat((total_X, X_dropout))
        total_y = torch.cat((total_y, total_y))

        print(f'Shape of X after channel dropout {total_X.shape}')
        print(f'Shape of Y: {total_y.shape}')

    if time_reverse != 0:
        mask = (torch.rand(total_X.shape[0]) >= time_reverse)
        X_reverse = torch.flip((total_X[mask]), [2])
        total_X = torch.cat((total_X, X_reverse))
        total_y = torch.cat((total_y, total_y[mask]))

        print(f'Shape of X after reverse {total_X.shape}')
        print(f'Shape of Y: {total_y.shape}')


    return total_X,total_y
    return total_X,total_y


def test_data_prep(X):

    total_X = None

    X = X[:,:,0:800]
    print('Shape of X after trimming:', X.shape)
    X_max, _ = torch.max(X.view(X.size(0), X.size(1), -1, 2), axis=3)

    total_X = X_max
    print('Shape of X after maxpooling:',total_X.shape)

    return total_X

# load data

In [6]:
## Loading the dataset
X_test = np.load("./project_data/X_test.npy")
y_test = np.load("./project_data/y_test.npy")-769
person_test = np.load("./project_data/person_test.npy")
X_train_valid = np.load("./project_data/X_train_valid.npy")
y_train_valid = np.load("./project_data/y_train_valid.npy")-769
person_train_valid = np.load("./project_data/person_train_valid.npy")
print("---all data---")
print("X_test shape\t\t", X_test.shape)
print("y_test shape\t\t", y_test.shape)
print("person_test shape\t", person_test.shape)
print("X_train_valid shape\t", X_train_valid.shape)
print("y_train_valid shape\t", y_train_valid.shape)
print("person_train_valid shape", person_train_valid.shape)
print()

---all data---
X_test shape		 (443, 22, 1000)
y_test shape		 (443,)
person_test shape	 (443, 1)
X_train_valid shape	 (2115, 22, 1000)
y_train_valid shape	 (2115,)
person_train_valid shape (2115, 1)



# select data (or subset of data)

In [7]:
# subjects is to load a subset of the data for a particular subject
# subjects = [0, 1, 2, 3, 4, 5, 6, 7, 8] #use 0-8 for subjects 1-9
subjects = [0] #use 0-8 for subjects 1-9

X_test_subs = np.empty((0, X_test.shape[1], X_test.shape[2]))
y_test_subs = np.empty((0))
X_train_valid_subs = np.empty((0, X_train_valid.shape[1], X_train_valid.shape[2]))
y_train_valid_subs = np.empty((0))

for i in subjects:
  sub = i
  X_test_subs = np.concatenate( (X_test_subs, X_test[np.where(person_test == sub)[0],:,:] ))
  y_test_subs = np.concatenate( (y_test_subs, y_test[np.where(person_test == sub)[0]] ))
  X_train_valid_subs = np.concatenate(( X_train_valid_subs, X_train_valid[np.where(person_train_valid == sub)[0], :, :] ))
  y_train_valid_subs = np.concatenate(( y_train_valid_subs, y_train_valid[np.where(person_train_valid == sub)[0]] ))

X_test = X_test_subs
y_test = y_test_subs
X_train_valid = X_train_valid_subs
y_train_valid = y_train_valid_subs
print("---after isolating subjects---")
print("X_test shape\t\t", X_test.shape)
print("y_test shape\t\t", y_test.shape)
print("X_train_valid shape\t", X_train_valid.shape)
print("y_train_valid shape\t", y_train_valid.shape)

---after isolating subjects---
X_test shape		 (50, 22, 1000)
y_test shape		 (50,)
X_train_valid shape	 (237, 22, 1000)
y_train_valid shape	 (237,)


In [8]:
input_size = 22
hidden_size = 256
num_layers = 10
num_classes = 4
learning_rate = 1e-4
batch_size = 200
num_epochs = 100
dropout = 0.3

folds = 10
split_seed = 1
subsample = 2
average = 2
channel_dropout = 0.2
time_reverse = 0.8
noise = True

X_train_valid = torch.Tensor(X_train_valid)
y_train_valid = torch.Tensor(y_train_valid)

print(f'SOME {X_train_valid.shape}')
indices = torch.randperm(X_train_valid.shape[0])
#print(indices)
split_idx = int(X_train_valid.shape[0] * ((folds-1)/folds))
#print(split_idx)
X_train, X_valid = X_train_valid[indices[:split_idx]], X_train_valid[indices[split_idx:]]
y_train, y_valid = y_train_valid[indices[:split_idx]], y_train_valid[indices[split_idx:]]
X_test, y_test = torch.Tensor(X_test), torch.Tensor(y_test)


print('Prepping Training Data')
X_train, y_train = data_prep(X_train, y_train, subsample, average, noise, channel_dropout, time_reverse)
print('\nPrepping Validation Data')
X_valid, y_valid = data_prep(X_valid, y_valid, subsample, average, noise, channel_dropout, time_reverse)
print('\nPrepping Test Data')
X_test = test_data_prep(X_test)
print('\nFINISHED PREP\n')

print('Final shape of training set:', X_train.shape)
print('Final shape of validation set:', X_valid.shape)
print('Final shape of test set:', X_test.shape)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_valid, y_valid), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

SOME torch.Size([237, 22, 1000])
Prepping Training Data
Shape of X after trimming: torch.Size([213, 22, 800])
Shape of X after maxpooling: torch.Size([213, 22, 400])
Shape of X after averaging+noise and concatenating: torch.Size([426, 22, 400])
torch.Size([426, 1])
torch.Size([213, 1])
torch.Size([639, 1])
torch.Size([213, 1])
Shape of X after subsampling and concatenating: torch.Size([852, 22, 400])
Shape of Y: torch.Size([852])
Shape of X after channel dropout torch.Size([1704, 22, 400])
Shape of Y: torch.Size([1704])
Shape of X after reverse torch.Size([2042, 22, 400])
Shape of Y: torch.Size([2042])

Prepping Validation Data
Shape of X after trimming: torch.Size([24, 22, 800])
Shape of X after maxpooling: torch.Size([24, 22, 400])
Shape of X after averaging+noise and concatenating: torch.Size([48, 22, 400])
torch.Size([48, 1])
torch.Size([24, 1])
torch.Size([72, 1])
torch.Size([24, 1])
Shape of X after subsampling and concatenating: torch.Size([96, 22, 400])
Shape of Y: torch.Size([

# **No Augment Models**

In [9]:
cnn_model = torch.load('./results-no-augment/cnn', map_location=torch.device('cuda'))
cnn_model.eval()

CNN(
  (conv1): Sequential(
    (0): Conv1d(22, 25, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(25, 50, kernel_size=(1, 25), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv1d(50, 100, kernel_size=(6,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv4): Sequential(
    (0): Conv1d(100, 405, kernel_size=(5,), stri

In [10]:
cnngru_model = torch.load('./results-no-augment/cgru', map_location=torch.device('cuda'))
cnngru_model.eval()

CGRU(
  (conv1): Sequential(
    (0): Conv1d(22, 25, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(25, 50, kernel_size=(1, 25), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv1d(50, 100, kernel_size=(6,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv4): Sequential(
    (0): Conv1d(100, 405, kernel_size=(5,), str

In [11]:
cnnlstm_model = torch.load('./results-no-augment/clstm', map_location=torch.device('cuda'))
cnnlstm_model.eval()

CLSTM(
  (conv1): Sequential(
    (0): Conv1d(22, 25, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(25, 50, kernel_size=(1, 25), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv1d(50, 100, kernel_size=(6,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv4): Sequential(
    (0): Conv1d(100, 405, kernel_size=(5,), st

In [12]:
lstm_model = torch.load('./results-no-augment/llstm', map_location=torch.device('cuda'))
lstm_model.eval()

LSTM_LSTM(
  (lstm1): LSTM(22, 64, num_layers=2, batch_first=True, bidirectional=True)
  (lstm2): LSTM(128, 64, num_layers=2, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=54, bias=True)
  (fc): Sequential(
    (0): Linear(in_features=54, out_features=44, bias=True)
    (1): BatchNorm1d(44, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=44, out_features=4, bias=True)
  )
)

In [13]:
gru_model = torch.load('./results-no-augment/gru', map_location=torch.device('cuda'))
gru_model.eval()

GRU(
  (gru): GRU(22, 64, num_layers=3, dropout=0.5, bidirectional=True)
  (linear): Linear(in_features=128, out_features=54, bias=True)
  (fc): Sequential(
    (0): Linear(in_features=54, out_features=44, bias=True)
    (1): BatchNorm1d(44, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=44, out_features=4, bias=True)
  )
)

In [14]:
shallowcnn_model = torch.load('./results-no-augment/ctransformer', map_location=torch.device('cuda'))
shallowcnn_model.eval()

CTransformer(
  (conv1): Sequential(
    (0): Conv1d(22, 25, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(25, 200, kernel_size=(1, 25), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (flatten): Flatten(start_dim=2, end_dim=-1)
  (encoding): Positional(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (l

# **No Augment Comparison**

In [15]:
test_model(cnn_model, test_loader,name="cnn")

cnn Test acc: 0.780


In [16]:
test_model(cnngru_model, test_loader,name="cgru")

cgru Test acc: 0.600


In [17]:
test_model(cnnlstm_model, test_loader,name="clstm")

clstm Test acc: 0.660


In [18]:
test_model(lstm_model, test_loader,name="llstm")

llstm Test acc: 0.360


In [19]:
test_model(shallowcnn_model, test_loader,name="ctransformer")

ctransformer Test acc: 0.660


# **No Augment Models**

In [20]:
cnn_model = torch.load('./results-augment/cnn', map_location=torch.device('cuda'))
cnn_model.eval()

CNN(
  (conv1): Sequential(
    (0): Conv1d(22, 25, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(25, 50, kernel_size=(1, 25), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv1d(50, 100, kernel_size=(6,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv4): Sequential(
    (0): Conv1d(100, 405, kernel_size=(5,), stri

In [21]:
cnngru_model = torch.load('./results-augment/cgru', map_location=torch.device('cuda'))
cnngru_model.eval()

CGRU(
  (conv1): Sequential(
    (0): Conv1d(22, 25, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(25, 50, kernel_size=(1, 25), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv1d(50, 100, kernel_size=(6,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv4): Sequential(
    (0): Conv1d(100, 405, kernel_size=(5,), str

In [22]:
cnnlstm_model = torch.load('./results-augment/clstm', map_location=torch.device('cuda'))
cnnlstm_model.eval()

CLSTM(
  (conv1): Sequential(
    (0): Conv1d(22, 25, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(25, 50, kernel_size=(1, 25), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv1d(50, 100, kernel_size=(6,), stride=(1,))
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv4): Sequential(
    (0): Conv1d(100, 405, kernel_size=(5,), st

In [23]:
lstm_model = torch.load('./results-augment/llstm', map_location=torch.device('cuda'))
lstm_model.eval()

LSTM_LSTM(
  (lstm1): LSTM(22, 64, num_layers=2, batch_first=True, bidirectional=True)
  (lstm2): LSTM(128, 64, num_layers=2, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=54, bias=True)
  (fc): Sequential(
    (0): Linear(in_features=54, out_features=44, bias=True)
    (1): BatchNorm1d(44, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=44, out_features=4, bias=True)
  )
)

In [24]:
gru_model = torch.load('./results-augment/gru', map_location=torch.device('cuda'))
gru_model.eval()

GRU(
  (gru): GRU(22, 64, num_layers=3, dropout=0.5, bidirectional=True)
  (linear): Linear(in_features=128, out_features=54, bias=True)
  (fc): Sequential(
    (0): Linear(in_features=54, out_features=44, bias=True)
    (1): BatchNorm1d(44, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=44, out_features=4, bias=True)
  )
)

In [25]:
shallowcnn_model = torch.load('./results-augment/ctransformer', map_location=torch.device('cuda'))
shallowcnn_model.eval()

CTransformer(
  (conv1): Sequential(
    (0): Conv1d(22, 25, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3647999634687216, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(25, 200, kernel_size=(1, 25), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3647999634687216, inplace=False)
  )
  (flatten): Flatten(start_dim=2, end_dim=-1)
  (encoding): Positional(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (l

# **Augment Comparison**

In [26]:
test_model(cnn_model, test_loader,name="cnn")

cnn Test acc: 0.740


In [27]:
test_model(cnngru_model, test_loader,name="cgru")

cgru Test acc: 0.660


In [28]:
test_model(cnnlstm_model, test_loader,name="clstm")

clstm Test acc: 0.680


In [29]:
test_model(lstm_model, test_loader,name="llstm")

llstm Test acc: 0.200


In [30]:
test_model(shallowcnn_model, test_loader,name="ctransformer")

ctransformer Test acc: 0.600
